# Телеграм бот с ИИ, анализирующий данные и визуализирующий их для пользователя

Цель работы: Проанализировать данные о продажах при помощи ии, визуализировать их для пользователя и внедрить это в телеграм бота.

## План работы

1. Открыть файл с данными, изучить общую информацию
2. Провести предобработку данных (проверка на пропуски, дубли, типы данных в столбцах)
3. Прописать промпты для g4f и нормализовать его работу с данными для анализа.
4. Подготовить функции создания графиков для дальнейших их запусков
5. Написать телеграм бота который будет принимать запросы, отправлять их ИИ и позже присылать ответ с png и html созданного графика. 

### Обработка данных

Для начала импортируем нужные библиотеки

In [ ]:
import re
import pandas as pd
import plotly.express as px
from g4f.client import Client
from ydata_profiling import ProfileReport
import telebot
import dataframe_image as dfi
from telebot import types

После указания библиотек нужно импортировать файл с которым мы будем работать. Сразу же проверим целостность его данных, их типы и количество.

In [ ]:
sales = pd.read_csv('ml_1. Прогнозирование спроса на товары в розничной торговле на основе данных о продажах и маркетинговой активности.csv')
sales.info()

В нашем файле отсутствуют пустующие столбцы, а также типы данных на первый взгляд логичны. Из этого делаем вывод, что удалять или форматировать ничего не нужно, поскольку целостность данных позволяет провести верный анализ.

Далее бегло осмотрим взятый датафрейм:

In [ ]:
sales.head(5)

Судя по имеющимся данным мы можем проанализировать количество продаж в соотношении каких-либо факторов.

Перед началом работы детальнее рассмотрим потенциально ненужный столбец:

In [ ]:
print(sales['Дата'].unique()) # Уникальные значения в столбце

По выводу кода мы видим, что уникальное значение всего одно. Смело избавляемся от данного столбца и упрощаем себе работу:

In [ ]:
sales = sales.drop(labels='Дата', axis=1)
sales

После проделанной работы можем подглядеть в ProfileReport дабы убедиться в правильности проделанных действий.

In [ ]:
profile = ProfileReport(sales, title="Pandas Profiling Report")

profile

По профилю, в первую очередь, мы можем заметить, что не оставили бесполезных столбцов, тем самым достаточно оптимизировав датафрейм. Но, к сожалению, по графику corellations и interactions человеческим взглядом трудно сделать какие либо выводы. Смеем предположить, что на графиках иного вида ситуация схожая, а потому воспользуемся помощью ИИ для решения этой задачи.

Поскольку далее мы будем использовать бесплатную версию gpt для анализа данных, нам следует их максимально сжать или обрезать. Пойдём по второму пути и будем анализировать каждую категорию по каким-либо критериям.

Раздробим наш датафрейм по категориям:

In [ ]:
grouped = sales.groupby('Категория товара')
categories = {category: group.reset_index(drop=True) for category, group in grouped} # создаём словарь с ключом имени категории и значением датафрейм

for category, group_df in categories.items():
    print(f"\nDataFrame для категории {category}:")
    print(group_df.head(5))

Мы получили датафреймы, но в них остались бесполезные столбцы "Категория товара". Мы уже указали наименования категорий датафреймов в словаре categories, а потому избавимся от этих столбцов:

In [ ]:
for category, group_df in categories.items():
    categories[category] = categories[category].drop(labels='Категория товара', axis=1)

for category, group_df in categories.items():
    print(f"\nDataFrame для категории {category}:")
    print(group_df)

На данном этапе мы имеем датафреймы готовые к работе с plotly и g4f. Пора переходить к следующему этапу работы...

### GPT for free нам в помощь!

Заранее создаём функцию для анализа данных с помощью ИИ

In [ ]:
def g4f_analyze(content:str, model='gpt-4o') -> str:
    client = Client()
    prompt = '''Тебе даётся датафрейм продаж. Проведи анализ и сделай вывод на русском языке, опиши всё словами. Бери в учёт все столбцы'''
    response = client.chat.completions.create(
        prompt=prompt,
        messages=[
        {
            "role": "system",
            "content": f'{prompt}'
        },
        {
            "role": "user",
            "content": f'{content}'
        }
    ])
    return response.choices[0].message.content

В качестве бонуса добавим функцию для ответа на (почти)любые вопросы, добавим промпт, дабы бот мог отвечать на вопросы по поводу функционала, а также напишем небольшую проверку полученного ответа и ограничим количество попыток генерации.

In [ ]:
def g4f_question(content:str, model='gpt-4o') -> str: 
    prompt = '''Ты искуственный интеллект для анализа данных датафрейма.
Ты должен отвечать только на русском языке в официальном стиле!

Твой функционал включает в себя:
1. Анализ данных.
2. Вывод результата анализа текстом без советов по написанию кода для этого анализа.
3. Вывод графиков по датафрейму в формате png или html.
4. Ответ пользователю на интересующие его вопросы по поводу твоего функционала, или другим темам.
'''
    i = 1
    while i <= 10:
        print(f'Попытка №{i}')
        client = Client()
        response = client.chat.completions.create(
            model=model,
            messages=[
                {
                    "role": "system",
                    "content": f'{prompt}'
                },
                {
                    "role": "user",
                    "content": f'{content}'
                }
            ]
        )
        res = response.choices[0].message.content
        i += 1
        clear_text = re.sub(r"[A-Za-z0-9]", "", res)
        if len(clear_text) > 25:
            if 'model' not in res.lower() and 'blackbox' not in res.lower():
                return res.strip()
    return 'Повторите попытку снова'

### Графики по нашим датафреймам

Напишем один большой блок кода с функциями для создания каждого возможного графика по желанию пользователя

In [ ]:
def create_graphic(category:str, tables:list) -> list:
    x, y, color = tables
    graph = px.scatter(categories[category], x=x, y=y, color=color)
    return [graph.to_image(), graph.to_html()]

def create_graphic_3d(category:str, tables:list) -> list:
    x,y,z = tables
    graph = px.scatter_3d(categories[category], x=x, y=y, z=z)
    return [graph.to_image(), graph.to_html()]

Мы конечно же не хотим писать целое приложение для нашего проекта, а потому используем наиболее доступную каждому пользователю платформу для взаимодействия с нашим проектом, а именно телеграм.

## Телеграм бот

Переходим к заключительной и самой интересной части нашей работы, но для начала заранее создадим png и html нашего основного датафрейма дабы не генерировать их каждый раз при необходимости показать пользователю.

In [ ]:
photo = sales.head(10)
sales.to_html("main_df.html", index=True)
dfi.export(photo,"main_df.png")

Остаётся только самое сложное - написание телеграм бота с полноценным меню. Лучшее что мы здесь сделаем, это используем состояние пользователя для создания меню с переходами дабы не обмазываться лишними операторами, а также напишем комментарии к каждому потенциально сложному блоку кода и не будем путать ни себя, ни людей которые будут работать с нашим кодом.

In [ ]:
TOKEN = '7677363072:AAGCj1dhXNqaH-j56lEgW3BJ1HoJx8W4kcw' # сюда вставить свой токен TG бота
bot = telebot.TeleBot(TOKEN)
user_states = {} # состояния пользователей для контролирования их передвижения по меню
category_global = ''


# СТАРТОВАЯ КОМАНДА # СТАРТОВАЯ КОМАНДА # СТАРТОВАЯ КОМАНДА # СТАРТОВАЯ КОМАНДА
@bot.message_handler(commands=['start'])
def start(message): # главная функция. тут мы создаём кнопки нашего меню и приветствуем пользователя
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Справка о боте")
    btn2 = types.KeyboardButton("Показать таблицу")
    btn3 = types.KeyboardButton("Начать работу с данными")
    btn4 = types.KeyboardButton("Задать вопрос ИИ")
    markup.add(btn1, btn2, btn3, btn4)
    
    txt = f"Привет, {message.from_user.first_name}! Я бот для анализа данных датафреймов, просмотрите мой функционал в командах."
    bot.send_message(message.chat.id, text=txt, reply_markup=markup)
    
    user_states[message.chat.id] = 'MAIN_MENU' # меняем состояние пользователя для перехода в другое меню
# СТАРТОВАЯ КОМАНДА # СТАРТОВАЯ КОМАНДА # СТАРТОВАЯ КОМАНДА # СТАРТОВАЯ КОМАНДА


# ГЛАВНОЕ МЕНЮ # ГЛАВНОЕ МЕНЮ # ГЛАВНОЕ МЕНЮ # ГЛАВНОЕ МЕНЮ
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'MAIN_MENU') # проверяем находится ли пользователь в нужном состоянии
def check_buttons(message): # главная функция
    """
    В данном блоке кода мы будем использовать операторы для проверки нажатия определённой кнопки путём прочтения сообщения пользователя.
    Справка нужна для краткого объяснения пользователю принципа работы бота и будет выводиться по желанию, ровно как и таблица основного датафрейма.
    Также мы реализуем переход пользователя между блоками меню, создавая новые кнопки и меняя состояние пользователя по id чата.
    """
    if message.text == 'Справка о боте': # вывод справки
        txt = '''   Я рад, что вы решили прочитать справку перед началом работы!

    Мой функционал включает в себя анализ и рисование графиков по тем данным что я имею.
К сожалению я являюсь демо-версией более гибкого в использовании бота, который должен принимать в себя любые данные
в формате excel или csv. Мой способ анализа основан на использовании g4f, что сильно ограничивает меня в работе,
потому в будущем я хотел бы иметь платную версию gpt для анализа более крупных датафреймов/датасетов.

Основные функции:
    1. Вывод датафреймов имеющихся в наличии в формате png и html
    2. Анализ выбранного датафрейма и вывод его графика в формате html или png(по выбору)
    3. Задать вопрос ИИ

Примечание:
    Анализ данных может занимать более 3 минут, а ответ вовсе вам не понравиться, поэтому, пожалуйста, будьте терпеливы и помните, что я могу сломаться
из-за частых запросов. Обязательно обождите обработки вашего запроса перед созданием нового!'''
        bot.send_message(message.chat.id, txt)

    elif message.text == 'Показать таблицу': # вывод таблицы в формате png и html
        bot.send_message(message.chat.id, "Отправляю вам обрезанный датафрейм в формате png и полный в формате html:")
        bot.send_photo(message.chat.id, open('main_df.png','rb'))
        bot.send_document(message.chat.id, open('main_df.html', 'rb'))

    elif message.text == 'Начать работу с данными': # переход в следующее меню. в дальнейшем для этого мы будем использовать функции
        user_states[message.chat.id] = 'CATEGORY_CHECK'
        
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton("Показать всю таблицу")
        btn2 = types.KeyboardButton("Показать категории")
        btn3 = types.KeyboardButton("Выбрать категорию для анализа")
        btn4 = types.KeyboardButton("Вернуться в главное меню")
        markup.add(btn1, btn2, btn3, btn4)
    
        txt = f"Вы перешли в меню работы с категориями"
        bot.send_message(message.chat.id, text=txt, reply_markup=markup)

    elif message.text == 'Задать вопрос ИИ': # небольшой бонус нашего бота в качестве ответа на вопросы
        user_states[message.chat.id] = 'QUESTIONS_PROCESSING'

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        back = types.KeyboardButton("Вернуться в главное меню")
        markup.add(back)
        bot.send_message(message.chat.id, 'Я готов отвечать на ваши вопросы!', reply_markup=markup)
# ГЛАВНОЕ МЕНЮ # ГЛАВНОЕ МЕНЮ # ГЛАВНОЕ МЕНЮ # ГЛАВНОЕ МЕНЮ       


# МЕНЮ ПРОСМОТРА КАТЕГОРИЙ # МЕНЮ ПРОСМОТРА КАТЕГОРИЙ
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'CATEGORY_CHECK')
def analyze_processing(message): # главная функция
    if message.text == 'Вернуться в главное меню':
        back_to_main_menu(message)

    elif message.text == 'Показать всю таблицу':
        bot.send_message(message.chat.id, "Отправляю вам обрезанный датафрейм в формате png и полный в формате html:")
        bot.send_photo(message.chat.id, open('main_df.png','rb'))
        bot.send_document(message.chat.id, open('main_df.html', 'rb'))

    elif message.text == 'Показать категории':
        text = 'Отправляю вам обрезанные png датафреймы категорий и полные в формате html:\n'
        bot.send_message(message.chat.id, text)
        for category, content in categories.items():
            bot.send_message(message.chat.id, category)
            photo = content.head(5)
            photo.to_html("sub_df.html", index=True)
            dfi.export(photo,"sub_df.png")
            bot.send_photo(message.chat.id, open('sub_df.png','rb'))
            bot.send_document(message.chat.id, open('sub_df.html', 'rb'))
    
    elif message.text == 'Выбрать категорию для анализа':
        go_to_choose_category_menu(message)
        

def back_to_main_menu(message): # вернуться в главное меню
    user_states[message.chat.id] = 'MAIN_MENU'
    start(message)
    
    
def go_to_choose_category_menu(message): # переход в меню выбора категории
    user_states[message.chat.id] = 'CATEGORY_CHOOSE'

    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Косметика")
    btn2 = types.KeyboardButton('Продукты питания')
    btn3 = types.KeyboardButton('Одежда')
    btn4 = types.KeyboardButton('Электроника')
    back = types.KeyboardButton("Вернуться в меню работы с категориями")
    markup.add(btn1, btn2, btn3, btn4, back)
    bot.send_message(message.chat.id, 'Вы перешли в меню выбора конкретной категории', reply_markup=markup)
# МЕНЮ ПРОСМОТРА КАТЕГОРИЙ # МЕНЮ ПРОСМОТРА КАТЕГОРИЙ


# ВЫБОР КАТЕГОРИИ # ВЫБОР КАТЕГОРИИ # ВЫБОР КАТЕГОРИИ
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'CATEGORY_CHOOSE')
def category_choose(message): # главная функция
    if message.text == 'Вернуться в меню работы с категориями':
        back_to_category_check_menu(message)

    elif message.text in ('Косметика','Продукты питания','Одежда','Электроника'):
        go_to_category_processing(message, message.text)


def back_to_category_check_menu(message): # вернуться  в меню работы с категориями
    user_states[message.chat.id] = 'CATEGORY_CHECK'
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Показать всю таблицу")
    btn2 = types.KeyboardButton("Показать категории")
    btn3 = types.KeyboardButton("Выбрать категорию для анализа")
    btn4 = types.KeyboardButton("Вернуться в главное меню")
    markup.add(btn1, btn2, btn3, btn4)

    txt = "Вы перешли в меню работы с категориями"
    
    bot.send_message(message.chat.id, text=txt, reply_markup=markup)
    analyze_processing(message)


def go_to_category_processing(message, category): # переход в меню анализа категории
    user_states[message.chat.id] = 'CATEGORY_PROCESSING'
    
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Показать датафрейм")
    btn2 = types.KeyboardButton("Нарисовать графики по датафрейму")
    btn3 = types.KeyboardButton("Выбрать 3 столбца для анализа")
    back = types.KeyboardButton("Вернуться в меню выбора категории")
    markup.add(btn1, btn2, btn3, back)

    global category_global
    category_global = category
    txt = f"Вы перешли в меню анализа категории '{category}'"
    bot.send_message(message.chat.id, text=txt, reply_markup=markup)
# ВЫБОР КАТЕГОРИИ # ВЫБОР КАТЕГОРИИ # ВЫБОР КАТЕГОРИИ 


# АНАЛИЗ КАТЕГОРИИ # АНАЛИЗ КАТЕГОРИИ # АНАЛИЗ КАТЕГОРИИ
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'CATEGORY_PROCESSING')
def category_processing(message): # главная функция
    if message.text == 'Вернуться в меню выбора категории':
        back_to_category_choose_menu(message)
        
    elif message.text == 'Показать датафрейм':
        bot.send_message(message.chat.id, category_global)
        photo = categories[category_global].head(5)
        photo.to_html("sub_df.html", index=True)
        dfi.export(photo,"sub_df.png")
        bot.send_photo(message.chat.id, open('sub_df.png','rb'))
        bot.send_document(message.chat.id, open('sub_df.html', 'rb'))
        
    elif message.text == 'Выбрать и нарисовать график по датафрейму':
        pass
    
    elif message.text == 'Выбрать 3 столбца для анализа':
        pass
    

def back_to_category_choose_menu(message): # возврат в меню выбора категории
    user_states[message.chat.id] = 'CATEGORY_CHOOSE'
    
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("Косметика")
    btn2 = types.KeyboardButton('Продукты питания')
    btn3 = types.KeyboardButton('Одежда')
    btn4 = types.KeyboardButton('Электроника')
    back = types.KeyboardButton("Вернуться в меню работы с категориями")
    markup.add(btn1, btn2, btn3, btn4, back)
    
    bot.send_message(message.chat.id, 'Вы перешли в меню выбора конкретной категории', reply_markup=markup)
    category_choose(message)
# АНАЛИЗ КАТЕГОРИИ # АНАЛИЗ КАТЕГОРИИ # АНАЛИЗ КАТЕГОРИИ


# ОТВЕТЫ ИИ НА ВОПРОСЫ # ОТВЕТЫ ИИ НА ВОПРОСЫ # ОТВЕТЫ ИИ НА ВОПРОСЫ
@bot.message_handler(func=lambda message: user_states.get(message.chat.id) == 'QUESTIONS_PROCESSING')
def questions_processing(message): # главная функция
    if message.text == "Вернуться в главное меню":
        back_to_main_menu2(message)
        
    else:
        bot.send_message(message.chat.id, text=g4f_question(message.text))


def back_to_main_menu2(message): # возврат в главное меню
    user_states[message.chat.id] = 'MAIN_MENU'
    start(message)
# ОТВЕТЫ ИИ НА ВОПРОСЫ # ОТВЕТЫ ИИ НА ВОПРОСЫ # ОТВЕТЫ ИИ НА ВОПРОСЫ


if __name__ == '__main__': # запускаем бота и отключаем ему возможность закончить работу
    bot.polling(none_stop=True)